<a href="https://colab.research.google.com/github/noeyhey/HUFS_Untructured-DM/blob/main/UnstructuredDM_0513.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q espnet== 202412
!pip install -q espnet_model_zoo

In [ ]:
!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187831595 sha256=58853b28a5a926cae14402bfd8d4d93a45ebf8f9e79533f37ab09d0d77a99c05
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash_attn


In [ ]:
import torch
from espnet2.bin.tts_inference import Text2Speech
import soundfile as sf
import IPython.display as ipd
from IPython.display import display, Audio
import time

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('cmudict')
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# @title Choose English model { run: "auto" }
lang = 'English'
tag = 'kan-bayashi/ljspeech_vits' #@param ["kan-bayashi/ljspeech_tacotron2", "kanbayashi/ljspeech_fastspeech", "kan-bayashi/ljspeech_fastspeech2", "kanbayashi/ljspeech_conformer_fastspeech2", "kanbayashi/ljspeech_joint_finetune_conformer_fastspeech2_hifigan", "kanbayashi/ljspeech_joint_train_conformer_fastspeech2_hifigan", "kan-bayashi/ljspeech_vits"]
{type:"string"}
vocoder_tag = "none" # @param ["none", "parallel_wavegan/ljspeech_parallel_wavegan.v1","parallel_wavegan/ljspeech_full_band_melgan.v2", "parallel_wavegan/ljspeech_multi_band_melgan.v2", "parallel_wavegan/ljspeech_hifigan.v1", "parallel_wavegan/ljspeech_style_melgan.v1"] {type:"string"}

In [ ]:
from espnet2.bin.tts_inference import Text2Speech
from espnet2.utils.types import str_or_none
text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag),
    vocoder_tag=str_or_none(vocoder_tag),
    device = "cuda" if torch.cuda.is_available() else "cpu",
    # Only for Tacotron 2 & Transformer
    threshold=0.5,
    # Only for Tacotron 2
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=False,
    backward_window=1,
    forward_window=3,
    # Only for FastSpeech & FastSpeech2 & VITS
    speed_control_alpha=1.0,
    # Only for VITS
    noise_scale=0.333,
    noise_scale_dur=0.333
    )

https://zenodo.org/record/5443814/files/tts_train_vits_raw_phn_tacotron_g2p_en_no_space_train.total_count.ave.zip?download=1: 100%|██████████| 356M/356M [03:12<00:00, 1.94MB/s]
/usr/local/lib/python3.11/dist-packages/espnet_model_zoo/downloader.py:364: UserWarning: Not validating checksum
  warnings.warn("Not validating checksum")
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.11/dist-packages/espnet2/gan_tts/vits/monotonic_align/__init__.py:19: UserWarning: Cython version is not available. Fallback to 'EXPERIMETAL' numba version. If you want to use the cython version, please build it as follows: `cd espnet2/gan_tts/vits/monotonic_align; python setup.py build_ext --inplace`
  warnings.warn(
 - discriminator_params.follow_official_norm
 - discriminator_params.scale_discriminator_param

In [ ]:
# 언어 설정 (예: 'en' 또는 'ko')
lang = "en" # 필요한 언어로 설정
# 사용자 입력 받기
print(f"Input your favorite sentence in {lang}.")
x = input()
# 음성 합성
try:
  with torch.no_grad():
    start = time.time()
    wav = text2speech(x)["wav"]
    rtf = (time.time() - start) / (len(wav) / text2speech.fs)
    print(f"RTF = {rtf:.5f}")

    # 오디오 재생
    display(Audio(wav.view(-1).cpu().numpy(), rate=text2speech.fs))

except Exception as e:
  print(f"Error occurred: {e}")

Input your favorite sentence in en.
I like you
RTF = 3.66427
